Alex Mei \\
CS 165B \\
Fall, 2021 \\

In [ ]:
# Imports
import numpy as np
import numpy.linalg as npla
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import random
import math
import time

In [ ]:
# Generate Data along the line y = x1 with noise in range(-0.5, 0.5)
# x2, ..., xn are all noise featurs in range (-0.5, 0.5)
# x0 is valued "1" for the w0 weight bias
def generateRegressionData(sampleSize: int, features: int):
  # 2 element tuple containing
  # param "sampleSize" lengthed list of feature rows containing [1, value in range(999.5, 1000.5), param "features" - 1 values in range(-0.5, 0.5)]
  # param"sampleSize" lengthed  list of labels with range (999.5, 1000.5) 
  return (
           [[1] + [10 + i + (random.random() - 0.5)] + [(random.random() - 0.5) for j in range(features - 1)] for i in range(sampleSize)],
           [10 + i + (random.random() - 0.5) for i in range(sampleSize)]
  )

In [ ]:
# Note: when regularization term = 0 (default), this is Least Squares Regression. Otherwise, it is Ridge Regression. 
def errorFunction(weights: np.array, data: np.array, labels: np.array, regularization: float = 0):
  return 0.5 * (npla.norm(data @ weights - labels, 2) ** 2 + regularization * npla.norm(weights) ** 2)

In [ ]:
# Note: when regularization term = 0 (default), this is Least Squares Regression. Otherwise, it is Ridge Regression. 
# Note: solution not guaranteed to exist when regularization term is zero
def closedFormSolver(data: np.array, labels: np.array, regularization: float = 0):
  return npla.inv(data.T @ data + regularization * np.identity(np.shape(data)[1])) @ data.T @ labels

In [ ]:
def gradientDescentStep(
    weights: np.array, 
    data: np.array, 
    labels: np.array, 
    stepSize: float, 
    batchSize: int = 0, 
    regularization: float = 0):
  datasetSize = np.shape(data)[0]

  # invalid batchSize, use batchSize = datasetSize
  if batchSize < 1 or batchSize >= datasetSize:
    selectedData = data
    selectedLabels = labels

  # else, randomly select batch of data given batch size
  else:
    choices = np.random.choice(datasetSize, size=batchSize, replace=False)
    selectedData = data[choices][:]
    selectedLabels = labels[choices]

  # compute gradient and direction vector
  gradient = selectedData.T @ selectedData @ weights - selectedData.T @ selectedLabels - regularization * weights
  direction = -1 * gradient / npla.norm(gradient, 2)

  # return update weights
  return weights + stepSize * direction

In [ ]:
def gradientDescent(
    weights: np.array, 
    data: np.array, 
    labels: np.array, 
    stepSize: float, 
    iterations: int,
    granularity: int, 
    plot: bool = False,
    graphTitle: str = "",
    batchSize: int = 0, 
    regularization: float = 0):
  
  plottingIterationNumber = list()
  plottingError = list()

  for i in range(1, iterations + 2):
    # reporting based on granularity 
    if i % granularity == 1:
      plottingIterationNumber.append(i)
      # plottingError.append(errorFunction(weights, data, labels, regularization))
      # plot the log error instead to be able to see a difference in the graphs
      plottingError.append(np.log(errorFunction(weights, data, labels, regularization)))

    # update weights
    weights = gradientDescentStep(weights, data, labels, stepSize, batchSize=batchSize, regularization=regularization)

  # convergence graph plot
  if plot:
    plt.plot(plottingIterationNumber, plottingError)

    # Plot configs
    plt.xlabel("Iteration Number")
    plt.ylabel("Objective Function")
    plt.title("Convergence Graph for {}".format(graphTitle))
    
    plt.show()

  return weights

In [ ]:
def normDistance(v1, v2):
  return npla.norm(v1 - v2, 2)

In [ ]:
def mse(predicted: np.array, actual: np.array):
  return npla.norm(predicted - actual, 2) ** 2 / np.shape(predicted)[0]

In [ ]:
# Q4. L2 Differences

# generate random data
data, labels = generateRegressionData(sampleSize=1000, features=50)
data = np.array(data)
labels = np.array(labels)
initialWeights = np.random.random(np.shape(data)[1])

# hyperparameters
regularization = 10 ** -6   # regularization term
stepSize = 10 ** -4         # learning rate
iterations = 10 ** 4        # maximum iterations
granularity = 10 ** 4       # plot reporting datapoint granularity

### Least Squares Regression
# Closed Form Solution
optimalWeights = closedFormSolver(data, labels)

# Gradient Descent
weights = initialWeights.copy()
gdWeights = gradientDescent(weights, data, labels, stepSize, iterations, granularity)

# Stochastic Gradient Descent
weights = initialWeights.copy()
sgdWeights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, batchSize=1)

print("{} Least Squares Regression {}".format("-" * 10, "-" * 10))
print("L2 Norm Difference of Weights between Gradient Descent & Stochastic Gradient Descent: {}".format(normDistance(gdWeights, sgdWeights)))
print("L2 Norm Difference of Weights between Gradient Descent & Closed Form Solution: {}".format(normDistance(gdWeights, optimalWeights)))
print("L2 Norm Difference of Weights between Stochastic Gradient Descent & Closed Form Solution: {}".format(normDistance(sgdWeights, optimalWeights)))

### Ridge Regression
# Closed Form Solution
optimalWeights = closedFormSolver(data, labels, regularization)

# Gradient Descent
weights = initialWeights.copy()
gdWeights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization)

# Stochastic Gradient Descent
weights = initialWeights.copy()
sgdWeights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization, batchSize=1)

print("{} Ridge Regression {}".format("-" * 10, "-" * 10))
print("L2 Norm Difference of Weights between Gradient Descent & Stochastic Gradient Descent: {}".format(normDistance(gdWeights, sgdWeights)))
print("L2 Norm Difference of Weights between Gradient Descent & Closed Form Solution: {}".format(normDistance(gdWeights, optimalWeights)))
print("L2 Norm Difference of Weights between Stochastic Gradient Descent & Closed Form Solution: {}".format(normDistance(sgdWeights, optimalWeights)))

In [ ]:
# Q5. Convergence Graphs

# generate random data
data, labels = generateRegressionData(sampleSize=1000, features=50)
data = np.array(data)
labels = np.array(labels)
initialWeights = np.random.random(np.shape(data)[1])

# hyperparameters
regularization = 10 ** -6   # regularization term
stepSize = 10 ** -4         # learning rate
iterations = 10 ** 4        # maximum iterations
granularity = 100           # number of iterations for early stopping consideration

# Least Squares Regression Gradient Descent
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, plot=True, graphTitle="Least Squares Regression Gradient Descent\n")
print(weights[0])

# Least Squares Regression Stochastic Gradient Descent
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, batchSize=1, plot=True, graphTitle="Least Squares Regression Stochastic Gradient Descent\n")
print(weights[0])

# Ridge Regression Gradient Descent
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization, plot=True, graphTitle="Ridge Regression Gradient Descent\n")
print(weights[0])

# Ridge Regression Stochastic Gradient Descent
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization, batchSize=1, plot=True, graphTitle="Ridge Regression Stochastic Gradient Descent\n")
print(weights[0])

# Printing weights[0] of each vector manually confirm they aren't exactly identical

In [ ]:
# Q6. Batched Gradient Descent

# generate random data
data, labels = generateRegressionData(sampleSize=5000, features=1000)
data = np.array(data)
labels = np.array(labels)
initialWeights = np.random.random(np.shape(data)[1])

# hyperparameters
regularization = 10 ** -6   # regularization term
stepSize = 10 ** -4         # learning rate
iterations = 10 ** 4        # maximum iterations
granularity = 100           # number of iterations for early stopping consideration

# Ridge Regression Gradient Descent Batch Size = 1
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization, batchSize=1, plot=True, graphTitle="Ridge Regression Gradient Descent Batch Size = 1\n")
print(weights[0])

# Ridge Regression Gradient Descent Batch Size = 10
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization, batchSize=10, plot=True, graphTitle="Ridge Regression Gradient Descent Batch Size = 10\n")
print(weights[0])


# Ridge Regression Gradient Descent Batch Size = 100
weights = initialWeights.copy()
weights = gradientDescent(weights, data, labels, stepSize, iterations, granularity, regularization=regularization, batchSize=100, plot=True, graphTitle="Ridge Regression Gradient Descent Batch Size = 100\n")
print(weights[0])

# Printing weights[0] of each vector manually confirm they aren't exactly identical

In [ ]:
# Q7a. Generalizion Performance with Increasing Sample Size

# generate random data
data, labels = generateRegressionData(sampleSize=1000, features=100)
data = np.array(data)
labels = np.array(labels)

# train - test split
trainingData = data[:900]
trainingLabels = data[:900]
testingData = data[900:]
testingLabels = data[900:]

# hyperparameters
regularization = 10 ** -6   # regularization term

# trainingSize vs MSE
trainingSize = list()
meanSquaredError = list()

for i in range(50, 901, 50):
  # Compute Closed Form Solution
  optimalWeights = closedFormSolver(trainingData[:i], trainingLabels[:i], regularization)

  # Compute MSE
  trainingSize.append(i)
  meanSquaredError.append(mse(testingData @ optimalWeights, testingLabels))

# convergence graph plot
plt.plot(trainingSize, meanSquaredError)

# Plot configs
plt.xlabel("Training Sample Size")
plt.ylabel("Mean Squared Error")
plt.title("Generalizion Performance with Increasing Sample Size")
    
plt.show()

print()

In [ ]:
# Q7b. Generalizion Performance with Increasing Dimensions

# generate random data
data, labels = generateRegressionData(sampleSize=1000, features=450)
data = np.array(data)
labels = np.array(labels)

# train - test split
trainingData = data[:900]
trainingLabels = data[:900]
testingData = data[900:]
testingLabels = data[900:]

# hyperparameters
regularization = 10 ** -6   # regularization term

# featureSize vs MSE
featureSize = list()
meanSquaredError = list()

for i in range(100, 451, 50):
  # Compute Closed Form Solution
  optimalWeights = closedFormSolver(trainingData[:,:i], trainingLabels, regularization)

  # Compute MSE
  featureSize.append(i)
  meanSquaredError.append(mse(testingData[:,:i] @ optimalWeights, testingLabels))

# convergence graph plot
plt.plot(featureSize, meanSquaredError)

# Plot configs
plt.xlabel("Feature Size")
plt.ylabel("Mean Squared Error")
plt.title("Generalizion Performance with Increasing Dimensions")
    
plt.show()

print()

In [ ]:
# Q8. Computational Complexity of Closed Form Solution

# generate random data
data, labels = generateRegressionData(sampleSize=1000, features=2000)
data = np.array(data)
labels = np.array(labels)

# hyperparameters
regularization = 10 ** -6   # regularization term

# featureSize vs MSE
featureSize = list()
computationTime = list()

for i in range(100, 2001, 100):
  t1 = time.time()

  # Compute Closed Form Solution
  optimalWeights = closedFormSolver(data[:,:i], labels, regularization)
  t2 = time.time()

  computationTime.append(t2 - t1)
  featureSize.append(i)

# convergence graph plot
plt.plot(featureSize, computationTime)

# Plot configs
plt.xlabel("Feature Size")
plt.ylabel("Computation Time")
plt.title("Computational Complexity of Closed Form Solution")
    
plt.show()

print()